In [7]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [11]:
import mediapipe as mp
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
def get_unique(connections):
    indices = set()
    for connection in connections:
        indices.update(connection)
    return sorted(indices)
peta_wajah = mp_face_mesh.FACEMESH_FACE_OVAL
peta_mulut = mp_face_mesh.FACEMESH_LIPS
peta_mata_luar = mp_face_mesh.FACEMESH_LEFT_EYE | mp_face_mesh.FACEMESH_RIGHT_EYE
peta_alis = mp_face_mesh.FACEMESH_LEFT_EYEBROW | mp_face_mesh.FACEMESH_RIGHT_EYEBROW
peta_mata_keseluruhan = peta_mata_luar
peta_wajah_keseluruhan = mp_face_mesh.FACEMESH_TESSELATION
peta_hidung = mp_face_mesh.FACEMESH_NOSE

peta_point4a = peta_hidung | peta_mata_luar | peta_alis | peta_mulut
peta_point4b = peta_wajah | peta_mata_luar | peta_alis | peta_mulut
peta_point4c = peta_wajah | peta_mata_luar | peta_hidung | peta_mulut
peta_point4d = peta_wajah | peta_alis | peta_hidung | peta_mulut

point4d = get_unique(peta_point4d)
point4c = get_unique(peta_point4c)
point4b = get_unique(peta_point4b)
point4a = get_unique(peta_point4a)
hidung = get_unique(peta_hidung)
mulut = get_unique(peta_mulut) 
wajah = get_unique(peta_wajah)
mata_luar = get_unique(peta_mata_luar)
alis = get_unique(peta_alis)
mata_keseluruhan = get_unique(peta_mata_keseluruhan)
wajah_penuh = get_unique(peta_wajah_keseluruhan)


In [29]:
def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_image)
    #M_A_W_H
    model = list(point4d)
    #print(model)
    if not results.multi_face_landmarks:
        return None
    landmarks = []
    height, width, _ = image.shape
    for face_landmarks in results.multi_face_landmarks:
        tanda = face_landmarks.landmark
        for index in model:
        # for lm in face_landmarks.landmark:
            x = int(tanda[index].x * width)
            y = int(tanda[index].y * height)
            landmarks.append([x,y])
    return np.array(landmarks).flatten()

In [30]:
data = []
nondata =[]
path_img = "images"
image_paths= os.listdir(path_img)
#print(image_paths)
i = 0
for img_path in image_paths:
    full_img_path = os.path.join(path_img, img_path)
    lm = extract_landmarks(full_img_path)
    if lm is not None:
        data.append(lm)
    else:
        nondata.append(i)
    i=i+1

In [31]:
print(nondata)

[1852, 1870, 1871, 1874, 1889, 1899, 1901, 1909, 1912, 1913, 1914, 1916, 1918, 1919, 1924, 1936, 1941, 1957, 1979, 1993, 2002, 2023, 2040, 2059, 2060, 2152, 2154, 2155, 2223, 2242, 2246, 2252, 2262, 2269, 2271, 2275, 2289, 2293, 2294, 2326, 2330, 2333, 2351, 2355, 2356, 2359, 2411, 2416, 2418, 2422, 2429, 2436, 2441, 2442, 2451, 2455, 2457, 2464, 2482, 2487, 2503, 2505, 2521, 2537, 2544, 2558, 2562, 2563, 2565, 2570, 2575, 2577, 2579, 2604, 2612, 2616, 2622, 2623, 2648, 2657, 2668, 2682, 2697, 2698, 2700, 2701, 2702, 2709, 2715, 2721, 2722, 2724, 2725, 2733, 2737, 2738, 2746, 2754, 2758, 2766, 2767, 2769, 2781, 2798, 2799, 2807, 2811, 2816, 2817, 2823, 2916, 2917, 3190, 3191, 3197, 3199, 3520, 3521, 3783, 4087, 4090, 4091, 4092, 4093, 4094, 4098, 4099, 4101, 4102, 4103, 4104, 4105, 4106, 4107, 4108, 4109, 4116, 4117, 4121, 4123, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4137, 4138, 4139, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4149, 4150, 4151, 4157, 4159, 4160, 4161, 416

In [32]:
def extract_lable(labels,path,nondata):
    l = []
    i_L = 0
    for label in labels :
        label = os.path.join(path,label)
        with open(label,"r") as data:
            readdata= data.readlines()
            obj = readdata[0][0]
            if nondata.count(i_L) == 0:
                l.append(obj)
        i_L = i_L+1
    return(l)
    
# # y = list(zip(*y))
# y = np.array(y)
# np.transpose(y)
# print(y)

In [33]:
path = "labels"
filenames= os.listdir(path)
labels = extract_lable(filenames,path,nondata)
print(len(data))
print(len(labels))
# X = pd.read_csv('landmarks_dataset(5).csv')
# X.head()

13780
13780


In [34]:
X = np.array(data)
y = np.array(labels)
# print(y)
 # print(y.size)

df = pd.DataFrame(X)
df['label'] = y
df.to_csv('landmarks_dataset_M_A_W_H.csv', index=False)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf']
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'metric':['euclidean', 'manhattan', 'minkowski'],
    'weights': ['uniform', 'distance']
}

svc = SVC()
knn = KNeighborsClassifier()


models = {
    'SVC': (svc, param_grid_svc),
    'KNN': (knn, param_grid_knn)
}


In [36]:
results = []
best_models = {}

for model_name, (model, param_grid) in models.items():
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_models[model_name] = grid_search.best_estimator_

    # Determine the number of splits
    num_splits = len([key for key in grid_search.cv_results_.keys() if key.startswith('split') and key.endswith('_test_score')])

    for i in range(len(grid_search.cv_results_['mean_test_score'])):
        params = grid_search.cv_results_['params'][i]
        mean_score = grid_search.cv_results_['mean_test_score'][i]
        mean_fit_time = grid_search.cv_results_['mean_fit_time'][i]
        mean_score_time = grid_search.cv_results_['mean_score_time'][i]
        total_training_time = mean_fit_time + mean_score_time

        # Collect split scores
        split_scores = {f'split{split_idx}_test_score': grid_search.cv_results_[f'split{split_idx}_test_score'][i] for split_idx in range(num_splits)}

        result = {
            'model': model_name,
            'params': params,
            'mean_test_score': mean_score,
            'mean_fit_time': mean_fit_time,
            'mean_score_time': mean_score_time,
            'total_training_time': total_training_time
        }
        result.update(split_scores)

        results.append(result)

    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best cross-validation accuracy for {model_name}: {grid_search.best_score_}")

results_df = pd.DataFrame(results)
results_df.to_excel('grid_search_results_M_A_W_H.xlsx', index=False)


Best parameters for SVC: {'C': 0.1, 'kernel': 'linear'}
Best cross-validation accuracy for SVC: 0.8400761493567288
Best parameters for KNN: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Best cross-validation accuracy for KNN: 0.927611546137987


In [ ]:
test_results = []

for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    test_results.append({
        'model': model_name,
        'accuracy': accuracy
    })
    print(f"Test accuracy for {model_name}: {accuracy}")

test_results_df = pd.DataFrame(test_results)
test_results_df.to_excel('test_results.xlsx', index=False)

plt.figure(figsize=(14, 7))
sns.lineplot(data=results_df, x='params', y='mean_test_score', hue='model', marker='o')
plt.xticks(rotation=90)
plt.title('Grid Search Mean Test Scores')
plt.xlabel('Hyperparameters')
plt.ylabel('Mean Test Score')
plt.legend(title='Model')
plt.tight_layout()
plt.savefig('grid_search_results.png')
plt.show()

plt.figure(figsize=(7, 5))
sns.barplot(data=test_results_df, x='model', y='accuracy', palette='viridis')
plt.title('Test Accuracies of Best Models')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig('test_results.png')
plt.show()